We consider the following simple 3-bus example network as a minimal example:

<img src="pics/3bus-system.png" width="50%">

The above network can be created in pandapower as follows:

In [2]:
import pandapower as pp

#create empty net
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=20., name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=0.4, name="Bus 2")
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")

#create bus elements
pp.create_ext_grid(net, bus=bus1, vm_pu=1.02, name="Grid Connection")
pp.create_load(net, bus=bus3, p_kw=100, q_kvar=50, name="Load")

#create branch elements
trafo = pp.create_transformer(net, hv_bus=bus1, lv_bus=bus2,
                              std_type="0.4 MVA 20/0.4 kV", name="Trafo")
line = pp.create_line(net, from_bus=bus2, to_bus=bus3, length_km=0.1,
                      std_type="NAYY 4x50 SE", name="Line")

Each dataframe in a pandapower net object contains the information about one pandapower element, such as line, load transformer etc.

In [4]:
net.bus

,name,vn_kv,type,zone,in_service
0,Bus 1,20.0,b,None,True
1,Bus 2,0.4,b,None,True
2,Bus 3,0.4,b,None,True


In [3]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,max_i_ka,df,parallel,type,in_service
0,Line,NAYY 4x50 SE,1,2,0.1,0.642,0.083,210.0,0.142,1.0,1,cs,True


In [5]:
net.trafo

,name,std_type,hv_bus,lv_bus,sn_kva,vn_hv_kv,vn_lv_kv,vsc_percent,vscr_percent,pfe_kw,...,tp_side,tp_mid,tp_min,tp_max,tp_st_percent,tp_st_degree,tp_pos,parallel,df,in_service
0,Trafo,0.4 MVA 20/0.4 kV,0,1,400.0,20.0,0.4,6.0,1.425,1.35,...,hv,0,-2,2,2.5,0.0,0,1,1.0,True


In [6]:
net.load

,name,bus,p_kw,q_kvar,const_z_percent,const_i_percent,sn_kva,scaling,in_service,type
0,Load,2,100.0,50.0,0.0,0.0,NaN,1.0,True,None


Note that line and transformer are created from the standard type library. The electric parameters of are automatically filled in from the standard type library.

We now run a power flow:

In [8]:
pp.runpp(net)

And look at the results:

In [11]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.020000,0.000000,-107.265391,-52.675195
1,1.008843,-0.760126,0.000000,0.000000
2,0.964431,0.115859,100.000000,50.000000


In [10]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,105.392391,50.696119,-100.0,-50.0,5.392391,0.696119,0.167325,0.167326,0.167326,117.835208


In [12]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,107.265391,52.675195,-105.392391,-50.696119,1.872999,1.979076,0.003382,0.167325,29.289513


In [15]:
net.trafo.tp_pos.at[trafo] = -1
pp.runpp(net)

Transformer loading: 29.22 percent
Line loading: 114.545 percent
Voltage magnitude vector: [ 1.02        1.03530089  0.99213495]


In [16]:
pp.create_switch(net, bus=bus3, element=line, et="l", closed=False)
pp.runpp(net)
print("Transformer loading: %.2f percent"%net.res_trafo.loading_percent.at[0])
print("Line loading: %.3f percent"%net.res_line.loading_percent.at[0])
print("Voltage magnitude vector: %s"%net.res_bus.vm_pu.values)

Transformer loading: 0.36 percent
Line loading: 0.001 percent
Voltage magnitude vector: [ 1.02        1.04612885         nan]


In [18]:
import pandapower.topology as top
top.unsupplied_buses(net)

{2}

In [27]:
net.switch.closed.at[0] = True
mg = top.create_nxgraph(net, include_trafos=False)
list(top.connected_component(mg, 2))

[2, 1]

In [33]:
import pandapower.shortcircuit as sc
net.ext_grid["s_sc_max_mva"] = 100
net.ext_grid["rx_max"] = 0.1
sc.calc_sc(net, case="max", ip=True, r_fault_ohm=2.)
print(net.res_bus_sc)

    ikss_ka     ip_ka
0  2.534707  4.317318
1  0.126631  0.182666
2  0.122698  0.176991
